In [1]:
from imdb import IMDb
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
import cPickle as pickle
ia = IMDb(accessSystem='http')
from collections import defaultdict 
import io
from datetime import datetime
import time
from sklearn.linear_model import LogisticRegression, ElasticNet
import sklearn.svm
import math
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
# Load AAdf (df of Oscar nominated movies)
AAdf = pickle.load(open('AAdictfinal.p','rb'))

In [4]:
AAdf.keys()

['0082894',
 '0158983',
 '0111503',
 '0090223',
 '0092136',
 '0181865',
 '0110057',
 '0094608',
 '0094606',
 '0085213',
 '0086147',
 '0083987',
 '0116754',
 '0933357',
 '0115753',
 '0088957',
 '0115751',
 '0084469',
 '0465551',
 '0118167',
 '0112573',
 '0089408',
 '0099121',
 '0113473',
 '0120338',
 '0151568',
 '0112579',
 '0443434',
 '0097635',
 '0105265',
 '0090605',
 '0387877',
 '0814075',
 '0093817',
 '0210853',
 '0086429',
 '0166943',
 '0088146',
 '0111579',
 '0093818',
 '0095647',
 '0117500',
 '0141963',
 '0096332',
 '0082416',
 '0096238',
 '0304107',
 '0101640',
 '0141560',
 '0119822',
 '0382330',
 '0118055',
 '0092229',
 '0303336',
 '0112883',
 '0343121',
 '0090319',
 '0092593',
 '0299178',
 '0082945',
 '0082869',
 '0088414',
 '0085482',
 '0190861',
 '0424024',
 '0090886',
 '0097441',
 '0315733',
 '0110357',
 '0120382',
 '0107822',
 '0095638',
 '0102368',
 '0099012',
 '0104275',
 '0095953',
 '0110428',
 '0316654',
 '0118632',
 '0280353',
 '0093105',
 '0175880',
 '0097118',
 '01